In [ ]:
import glob
from spf.dataset.spf_dataset import v5spfdataset

rx_idx = 1
v4_files = glob.glob("/mnt/4tb_ssd/nosig_data/*v4_cal*.zarr")
v4_dss = [
    v5spfdataset(
        prefix=v4_file,
        nthetas=65,
        ignore_qc=True,
        precompute_cache="/mnt/4tb_ssd/precompute_cache_new/",
        gpu=False,
        skip_fields=["signal_matrix"],
        snapshots_per_session=1,
        n_parallel=4,
    )
    for v4_file in v4_files
]

In [ ]:
print("Using %d files as input" % len(v4_dss))

In [ ]:
import tqdm

# read all the files and find the min and max for x (mm) and y (mm)
x_min, x_max = None, None
y_min, y_max = None, None

for v4_ds in v4_dss:

    _x_min, _x_max = (
        v4_ds.cached_keys[rx_idx]["rx_pos_mm"][:, 0].min(),
        v4_ds.cached_keys[rx_idx]["rx_pos_mm"][:, 0].max(),
    )
    _y_min, _y_max = (
        v4_ds.cached_keys[rx_idx]["rx_pos_mm"][:, 1].min(),
        v4_ds.cached_keys[rx_idx]["rx_pos_mm"][:, 1].max(),
    )
    if x_min is None:
        x_min = _x_min
        x_max = _x_max
        y_min = _y_min
        y_max = _y_max
    else:
        x_min = min(x_min, _x_min)
        x_max = max(x_max, _x_max)
        y_min = min(y_min, _y_min)
        y_max = max(y_max, _y_max)

In [ ]:
v4_dss[0].mean_phase

In [ ]:
import numpy as np


# split the area into a grid of bin_size
def coord_to_bin(x, y, bin_size=5):
    return int(x // bin_size), int(y // bin_size)


bin_size = 30
max_x, max_y = coord_to_bin(x_max, y_max, bin_size=bin_size)

means = np.zeros((int(max_x + 1), int(max_y + 1)))
counts = np.zeros((int(max_x + 1), int(max_y + 1)))
eps = 0  # 1e-3
# find the mean estimate angle for each bin
for v4_ds in v4_dss:
    mean_phases = v4_ds.mean_phase[f"r{rx_idx}"]
    for row_idx in range(v4_ds.cached_keys[rx_idx]["rx_pos_mm"].shape[0]):
        x, y = v4_ds.cached_keys[rx_idx]["rx_pos_mm"][row_idx]
        _x, _y = coord_to_bin(x, y, bin_size=bin_size)
        means[_x, _y] += mean_phases[row_idx]
        counts[_x, _y] += 1
means /= counts + eps

In [ ]:
# plt.hist(means.reshape(-1))

In [ ]:
var = np.zeros((int(max_x + 1), int(max_y + 1)))

# find the variance for each bin
for v4_ds in v4_dss:
    mean_phases = v4_ds.mean_phase[f"r{rx_idx}"]
    for row_idx in range(v4_ds.cached_keys[rx_idx]["rx_pos_mm"].shape[0]):
        x, y = v4_ds.cached_keys[rx_idx]["rx_pos_mm"][row_idx]
        _x, _y = coord_to_bin(x, y, bin_size=bin_size)
        var[_x, _y] += np.power(means[_x, _y] - mean_phases[row_idx], 2)
var /= counts + eps

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(45, 22))

im = axs[0].imshow(
    means.T * 360 / (2 * np.pi),
    origin="upper",
    extent=(0, means.shape[0], means.shape[1], 0),
    cmap="brg",
    aspect="auto",
)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax, orientation="vertical")
axs[0].invert_xaxis()

im = axs[1].imshow(np.clip(var.T * 360 / (2 * np.pi), a_max=30, a_min=0))

divider = make_axes_locatable(axs[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax, orientation="vertical")
axs[1].invert_xaxis()

# how many ticks to put on the sides
nticks = 7
xs = np.arange(0, means.shape[0], means.shape[0] // nticks)
ys = np.arange(0, means.shape[1], means.shape[1] // nticks)

axs[0].set_title("Radio angle estimate (degrees) vs position")
axs[1].set_title("Variance (degrees) vs position")
plt.tight_layout()
plt.savefig("%dmm_res_map.png" % bin_size)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(counts.reshape(-1))